In [7]:
with open("wine.txt", encoding='utf-8') as f:
    real_estate_sales = f.read()

In [8]:
from langchain.text_splitter import CharacterTextSplitter

In [9]:
text_splitter = CharacterTextSplitter(        
    separator = r'\d+\.',
    chunk_size = 100,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = True,
)

In [10]:
docs = text_splitter.create_documents([real_estate_sales])

In [12]:
docs[2]

Document(page_content='客户问题：白酒能保存多久？\n销售回答：如果储存得当，这款白酒可以保存多年而不失其风味。\n\n\\d+\\. 客户问题：这款白酒口感如何？\n销售回答：这款白酒口感醇厚，余味悠长，非常适合搭配各种菜肴。', metadata={})

In [13]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, OpenAIEmbeddings())

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', ConnectionResetError(10054, '远程主机强迫关闭了一个现有的连接。', None, 10054, None)).
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', ConnectionResetError(10054, '远程主机强迫关闭了一个现有的连接。', None, 10054, None)).


In [14]:
query = "酒好喝吗"

In [15]:
answer_list = db.similarity_search(query)

In [16]:
for ans in answer_list:
    print(ans.page_content + "\n")

客户问题：适合什么场合喝？
销售回答：这款白酒非常适合家庭聚餐、商务宴请或节日庆典等多种场合。

客户问题：我怎么知道这款酒适不适合我？
销售回答：我们有专门的品酒师，他们可以根据您的口味和需求，为您推荐合适的产品。

客户问题：你们有没有年份酒？
销售回答：是的，我们有限量的年份酒，每一瓶都有其独特的风味和纪念价值。

客户问题：这个酒怎么配餐最佳？
销售回答：这款白酒尤为适合搭配红肉和辛辣食物，能够衬托出酒和食物的风味。



In [17]:
topK_retriever = db.as_retriever(search_kwargs={"k": 3})

In [18]:
topK_retriever

VectorStoreRetriever(tags=['FAISS'], metadata=None, vectorstore=<langchain.vectorstores.faiss.FAISS object at 0x0000018B607DD9D0>, search_type='similarity', search_kwargs={'k': 3})

In [19]:
docs = topK_retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

客户问题：适合什么场合喝？
销售回答：这款白酒非常适合家庭聚餐、商务宴请或节日庆典等多种场合。

客户问题：我怎么知道这款酒适不适合我？
销售回答：我们有专门的品酒师，他们可以根据您的口味和需求，为您推荐合适的产品。

客户问题：你们有没有年份酒？
销售回答：是的，我们有限量的年份酒，每一瓶都有其独特的风味和纪念价值。



In [20]:
docs = topK_retriever.get_relevant_documents("你们有没有1万的酒啊？")

In [21]:
for doc in docs:
    print(doc.page_content + "\n")

客户问题：你们有没有年份酒？
销售回答：是的，我们有限量的年份酒，每一瓶都有其独特的风味和纪念价值。

客户问题：你们的白酒有几种型号？
销售回答：我们有多种型号和口感，满足不同消费者的需求。

客户问题：你们还有其他产品吗？
销售回答：除了白酒，我们还提供一系列与白酒相关的周边产品，如特制酒杯和礼品套装。

